In [ ]:
from agent_framework import MCPStdioTool, MCPStreamableHTTPTool, MCPStdioTool


In [ ]:
tool = MCPStreamableHTTPTool(
            name="Microsoft Learn MCP",
            url="https://learn.microsoft.com/api/mcp",
            # we don't require approval for microsoft_docs_search tool calls
            # but we do for any other tool
            # approval_mode={"never_require_approval": ["microsoft_docs_search"]},
        )

In [ ]:
tool2 = MCPStdioTool(  
    name="filesystem",  
    command="npx",  
    args=["-y", "@modelcontextprotocol/server-filesystem", r"C:\Users\blain\Documents"],  
    description="File system operations",  
)

In [ ]:
tool3 = MCPStreamableHTTPTool(
            name="localhost MCP",
            url="http://localhost:1337/mcp",
        )


In [ ]:
import logging
import httpx
import json

# Enable debug logging for HTTP requests
logging.basicConfig(level=logging.DEBUG, format='%(name)s - %(levelname)s - %(message)s')
httpx_logger = logging.getLogger("httpx")
httpx_logger.setLevel(logging.DEBUG)

# Also enable openai library logging if available
logging.getLogger("openai").setLevel(logging.DEBUG)

# Patch httpx to log request bodies
original_request = httpx.AsyncClient.request

async def logged_request(self, method, url, **kwargs):
    print(f"\n{'='*60}")
    print(f"HTTP {method} Request to: {url}")
    print(f"{'='*60}")
    
    if 'json' in kwargs:
        print("\nRequest Body (JSON):")
        print(json.dumps(kwargs['json'], indent=2))
    elif 'content' in kwargs:
        print("\nRequest Body (Content):")
        print(kwargs['content'])
    
    if 'headers' in kwargs:
        print("\nRequest Headers:")
        for key, value in kwargs['headers'].items():
            print(f"  {key}: {value}")
    
    print(f"{'='*60}\n")
    
    response = await original_request(self, method, url, **kwargs)
    
    print(f"\n{'='*60}")
    print(f"HTTP Response Status: {response.status_code}")
    print(f"{'='*60}\n")
    
    return response

httpx.AsyncClient.request = logged_request

from agent_framework.openai import OpenAIChatClient

# tool = MCPStreamableHTTPTool(
#             name="localhost MCP",
#             url="http://localhost:1337/mcp",
#         )
llm = llm = OpenAIChatClient(
        api_key="ollama", # Just a placeholder, Ollama doesn't require API key
        # base_url="http://localhost:11434/v1",
        base_url="http://ollama.home/v1",
        model_id="gpt-oss:20b",
    )
agent = llm.create_agent(
        name="test_agent",
        instructions="You are a helpful agent. You use Model Context Protocol (MCP) tools to answer user questions. "\
            "You can only respond using the tools available to you. Do not make up tool functionality. The tools will be" \
                "Provided to you in the prompt.",
        tools=[tool, tool3],
    )

query = "What MCP tools do you have?"
print(f"User: {query}")
print("\n=== HTTP Request/Response Details Below ===\n")
result = await agent.run(query)
print(f"\n=== End of HTTP Details ===\n")
print(f"Agent: {result}\n")


In [ ]:
import asyncio
import os
from random import randint
from typing import Annotated

from agent_framework.openai import OpenAIChatClient

"""
Ollama with OpenAI Chat Client Example

This sample demonstrates using Ollama models through OpenAI Chat Client by
configuring the base URL to point to your local Ollama server for local AI inference.
Ollama allows you to run large language models locally on your machine.

Environment Variables:
- OLLAMA_ENDPOINT: The base URL for your Ollama server (e.g., "http://localhost:11434/v1/v1/")
- OLLAMA_MODEL: The model name to use (e.g., "mistral", "llama3.2", "phi3")
"""


def get_weather(
    location: Annotated[str, "The location to get the weather for."],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."


async def non_streaming_example() -> None:
    """Example of non-streaming response (get the complete result at once)."""
    print("=== Non-streaming Response Example ===")

    agent = llm.create_agent(
        name="WeatherAgent",
        instructions="You are a helpful weather agent.",
        tools=get_weather,
    )

    query = "What's the weather like in Seattle?"
    print(f"User: {query}")
    result = await agent.run(query)
    print(f"Agent: {result}\n")


async def streaming_example() -> None:
    """Example of streaming response (get results as they are generated)."""
    print("=== Streaming Response Example ===")

    agent = llm.create_agent(
        name="WeatherAgent",
        instructions="You are a helpful weather agent.",
        tools=get_weather,
    )

    query = "What's the weather like in Portland?"
    print(f"User: {query}")
    print("Agent: ", end="", flush=True)
    async for chunk in agent.run_stream(query):
        if chunk.text:
            print(chunk.text, end="", flush=True)
    print("\n")

In [ ]:
await streaming_example()